In [14]:
%load_ext sql
%config SqlMagic.displaylimit = None

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
%sql sqlite:///cssys.db

In [16]:
%%sql
drop table if exists streams;
create table streams (
    user_id text,
    track_id text,
    timestamp text
);

# Drop and create the `track` table
%%sql
drop table if exists track;
create table track (
    track_id text,
    track_artist_ids text,
    track_title text
);

# Drop and create the `artist` table
%%sql
drop table if exists artist;
create table artist (
    artist_id text,
    artist_name text,
    follower_count integer,
    date text
);

# Insert sample data into the `streams` table
%%sql
insert into streams (user_id, track_id, timestamp) values
    ('U1', 'T1', '2025-01-01T00:01:00Z'),
    ('U1', 'T2', '2025-01-01T00:02:00Z'),
    ('U2', 'T1', '2025-01-01T00:03:00Z'),
    ('U3', 'T3', '2025-01-01T00:04:00Z'),
    ('U4', 'T2', '2025-01-01T00:05:00Z'),
    ('U4', 'T3', '2025-01-01T00:06:00Z');

# Insert sample data into the `track` table
%%sql
insert into track (track_id, track_artist_ids, track_title) values
    ('T1', 'A1', 'ABC'),
    ('T2', 'A1, A2', 'ABC - Remix'),
    ('T3', 'A2', 'XYZ');

# Insert sample data into the `artist` table
%%sql
insert into artist (artist_id, artist_name, follower_count, date) values
    ('A1', 'Artist A', 100000, '2025-01-01'),
    ('A2', 'Artist B', 150000, '2025-01-01');

# SQL Query to find overlapping fanbases
%%sql
WITH jan_streams AS (
    SELECT user_id, track_id
    FROM streams
    WHERE timestamp >= '2025-01-01' AND timestamp < '2025-02-01'
),
track_artists AS (
    SELECT track_id, TRIM(UNNEST(STRING_TO_ARRAY(track_artist_ids, ','))) AS artist_id
    FROM track
),
user_artist_stream_counts AS (
    SELECT js.user_id, ta.artist_id, COUNT(*) AS stream_count
    FROM jan_streams js
    JOIN track_artists ta ON js.track_id = ta.track_id
    GROUP BY js.user_id, ta.artist_id
),
artist_pairs AS (
    SELECT 
        ua1.artist_id AS artist_1_id,
        ua2.artist_id AS artist_2_id,
        COUNT(DISTINCT ua1.user_id) AS common_listener,
        SUM(ua1.stream_count) AS artist_1_streams_by_common_listener,
        SUM(ua2.stream_count) AS artist_2_streams_by_common_listener
    FROM user_artist_stream_counts ua1
    JOIN user_artist_stream_counts ua2 
    ON ua1.user_id = ua2.user_id
    WHERE ua1.artist_id < ua2.artist_id
    GROUP BY ua1.artist_id, ua2.artist_id
)
SELECT
    ap.artist_1_id,
    a1.artist_name AS artist_1_name,
    ap.artist_2_id,
    a2.artist_name AS artist_2_name,
    ap.common_listener,
    ap.artist_1_streams_by_common_listener,
    ap.artist_2_streams_by_common_listener
FROM artist_pairs ap
LEFT JOIN artist a1 ON ap.artist_1_id = a1.artist_id
LEFT JOIN artist a2 ON ap.artist_2_id = a2.artist_id
ORDER BY ap.artist_1_id, ap.artist_2_id;

 * sqlite:///cssys.db
Done.
Done.
(sqlite3.OperationalError) unrecognized token: "#"
[SQL: # Drop and create the `track` table
%%sql
drop table if exists track;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
%%sql
WITH jan_streams AS (
    SELECT
        user_id,
        track_id
    FROM
        streams
    WHERE
        timestamp >= '2025-01-01'
        AND timestamp < '2025-02-01'
),

track_artists AS (
    SELECT
        track_id,
        TRIM(UNNEST(STRING_TO_ARRAY(track_artist_ids, ','))) AS artist_id
    FROM
        track
),

user_artist_streams AS (
    SELECT DISTINCT js.user_id, ta.artist_id
    FROM jan_streams js JOIN track_artists ta ON js.track_id = ta.track_id
),

artist_pairs AS (
    SELECT ua1.artist_id AS artist_1_id, ua2.artist_id AS artist_2_id, COUNT(DISTINCT ua1.user_id) AS common_listener
    FROM
        user_artist_streams ua1
    JOIN
        user_artist_streams ua2
    ON
        ua1.user_id = ua2.user_id
    WHERE
        ua1.artist_id < ua2.artist_id
    GROUP BY
        ua1.artist_id,
        ua2.artist_id
)

SELECT
    ap.artist_1_id,
    a1.artist_name AS artist_1_name,
    ap.artist_2_id,
    a2.artist_name AS artist_2_name,
    ap.common_listener
FROM
    artist_pairs ap
LEFT JOIN
    artist a1
ON
    ap.artist_1_id = a1.artist_id
    AND a1.date <= '2025-01-31'
LEFT JOIN
    artist a2
ON
    ap.artist_2_id = a2.artist_id
    AND a2.date <= '2025-01-31'
ORDER BY
    ap.artist_1_id,
    ap.artist_2_id;

